In [8]:
import os
import numpy as np
import pandas as pd
import pickle

## Read in Core Covariates CSV

So really we'd only want to do this once we've fit the model.

In [74]:
datadir = '/Volumes/Transcend/birt data/eBird raw data'

file = 'srd_point_data_30km_v3.0.csv'
path_to_file = os.path.join(datadir, 'srd_point_data_30km_v3.0.csv')

# Missing values seem to be encoded as "?", so we're going to add this to the na_values argument.
srd30km = pd.read_csv(path_to_file, na_values = '?')

In [75]:
srd30km.columns = map(str.lower, srd30km.columns)

In [76]:
srd30km.shape

(130751, 496)

In [77]:
srd30km

,decimal_latitude,decimal_longitude,pop00_sqmi,housing_density,housing_percent_vacant,elev_gt,elev_ned,subnational2_code,bcr,bailey_ecoregion,...,nlcd2006_fs_c82_7500_pd,nlcd2006_fs_c82_7500_pland,nlcd2006_fs_c90_7500_ed,nlcd2006_fs_c90_7500_lpi,nlcd2006_fs_c90_7500_pd,nlcd2006_fs_c90_7500_pland,nlcd2006_fs_c95_7500_ed,nlcd2006_fs_c95_7500_lpi,nlcd2006_fs_c95_7500_pd,nlcd2006_fs_c95_7500_pland
0,30.517135,-104.836070,0.2,0.111695,0.426621,1006.0,999.61,US-TX-377,35.0,-321A,...,0.0000,0.0000,3.5391,0.7469,0.1472,1.8519,0.4161,0.0153,0.0566,0.0698
1,30.614037,-104.778619,0.3,0.207974,0.446701,1097.0,1109.49,US-TX-243,35.0,-321A,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,30.504075,-104.178670,0.3,0.207974,0.446701,1785.0,1709.06,US-TX-243,35.0,-321A,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,30.660314,-104.446625,0.3,0.207974,0.446701,1424.0,1420.24,US-TX-243,35.0,-321A,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,30.530671,-104.276839,0.3,0.207974,0.446701,1569.0,1580.80,US-TX-243,35.0,-321A,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
5,30.530026,-104.460528,0.3,0.207974,0.446701,1363.0,1368.15,US-TX-243,35.0,-321A,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
6,30.628577,-104.404305,0.3,0.207974,0.446701,1454.0,1444.11,US-TX-243,35.0,-321A,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7,30.588918,-104.524444,0.3,0.207974,0.446701,1342.0,1336.57,US-TX-243,35.0,-321A,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
8,30.606803,-104.405874,0.3,0.207974,0.446701,1434.0,1436.01,US-TX-243,35.0,-321A,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9,30.582815,-104.388503,0.3,0.207974,0.446701,1440.0,1446.03,US-TX-243,35.0,-321A,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [79]:
zen = pickle.load(open('/Volumes/Transcend/birt data/zen_regr.p', 'rb'))
zen_df = pickle.load(open('/Volumes/Transcend/birt data/zen_df.p', 'rb'))

In [83]:
X_names = list(zen_df.columns[0:-1])
keeps = srd30km.columns.isin(X_names)
srd = srd30km.loc[:, srd30km.columns.isin(X_names)]

In [84]:
srd.shape

(130751, 40)

In [95]:
srd.dtypes

pop00_sqmi                    float64
housing_density               float64
housing_percent_vacant        float64
elev_gt                       float64
elev_ned                      float64
bcr                           float64
bailey_ecoregion               object
omernik_l3_ecoregion          float64
nlcd2001_fs_c11_7500_pland    float64
nlcd2001_fs_c12_7500_pland    float64
nlcd2001_fs_c21_7500_pland    float64
nlcd2001_fs_c22_7500_pland    float64
nlcd2001_fs_c23_7500_pland    float64
nlcd2001_fs_c24_7500_pland    float64
nlcd2001_fs_c31_7500_pland    float64
nlcd2001_fs_c41_7500_pland    float64
nlcd2001_fs_c42_7500_pland    float64
nlcd2001_fs_c43_7500_pland    float64
nlcd2001_fs_c52_7500_pland    float64
nlcd2001_fs_c71_7500_pland    float64
nlcd2001_fs_c81_7500_pland    float64
nlcd2001_fs_c82_7500_pland    float64
nlcd2001_fs_c90_7500_pland    float64
nlcd2001_fs_c95_7500_pland    float64
nlcd2006_fs_c11_7500_pland    float64
nlcd2006_fs_c12_7500_pland    float64
nlcd2006_fs_

In [86]:
X_pred = np.array(srd)

In [87]:
X_pred

array([[0.2, 0.11169460465769014, 0.4266211604095565, ..., 0.0, 1.8519,
        0.0698],
       [0.3, 0.2079736374728546, 0.4467005076142129, ..., 0.0, 0.0, 0.0],
       [0.3, 0.2079736374728546, 0.4467005076142129, ..., 0.0, 0.0, 0.0],
       ..., 
       [0.4, 0.16850503076067933, 0.12244897959183666, ..., 55.8432,
        1.9292, 0.066],
       [0.4, 0.16850503076067933, 0.12244897959183666, ..., 53.4388,
        1.5864, 0.19399999999999998],
       [0.4, 0.16850503076067933, 0.12244897959183666, ...,
        27.055999999999997, 1.0776, 0.4]], dtype=object)

In [88]:
zen.predict(X_pred)

ValueError: could not convert string to float: '-331D'

In [24]:
# Joining method one: we remove cols in covariates which aren't in `migrations`.
cov_samp2 = cov_samp.loc[:, cov_samp.columns.isin(core_covariates)]
pd.concat([cov_samp2, zenaida_macroura_df], join = 'outer')

,_id,bailey_ecoregion,bcr,caus_prec,caus_snow,caus_temp_avg,caus_temp_max,caus_temp_min,elev_gt,elev_ned,...,nlcd2006_fs_c43_7500_pland,nlcd2006_fs_c52_7500_pland,nlcd2006_fs_c71_7500_pland,nlcd2006_fs_c81_7500_pland,nlcd2006_fs_c82_7500_pland,nlcd2006_fs_c90_7500_pland,nlcd2006_fs_c95_7500_pland,omernik_l3_ecoregion,pop00_sqmi,zenaida_macroura
67743,NaN,-212F,13.0,NaN,NaN,NaN,NaN,NaN,310.0,310.55,...,7.2500,5.9248,0.7676,31.5032,21.9404,4.3096,0.2968,83.0,49.8,NaN
76056,NaN,-331G,17.0,NaN,NaN,NaN,NaN,NaN,972.0,982.21,...,0.0000,34.0616,63.6876,0.0000,1.0732,0.5096,0.3612,43.0,0.5,NaN
97231,NaN,M221D,28.0,NaN,NaN,NaN,NaN,NaN,91.0,104.22,...,1.5828,0.0000,0.0068,38.7612,2.8116,0.7840,0.0860,66.0,72.9,NaN
103640,NaN,-331G,17.0,NaN,NaN,NaN,NaN,NaN,1218.0,1231.32,...,0.0000,2.5516,49.2472,0.0628,41.9392,2.4564,0.0204,43.0,1.0,NaN
39946,NaN,-222E,24.0,NaN,NaN,NaN,NaN,NaN,211.0,210.74,...,0.3876,0.0380,5.2112,22.7740,1.1776,0.0000,0.0180,71.0,105.4,NaN
8414,NaN,-221H,28.0,NaN,NaN,NaN,NaN,NaN,549.0,542.01,...,6.8400,5.6288,7.4204,6.7540,0.7512,0.3672,0.0000,68.0,46.5,NaN
60517,NaN,-234A,27.0,NaN,NaN,NaN,NaN,NaN,80.0,82.58,...,0.3740,9.2892,0.0716,11.3528,37.8564,6.4500,1.1856,74.0,100.2,NaN
40903,NaN,M331F,16.0,NaN,NaN,NaN,NaN,NaN,1889.0,1900.21,...,0.0212,42.1576,18.5436,0.3608,0.0000,0.5536,0.0204,21.0,5.5,NaN
50351,NaN,-322A,33.0,NaN,NaN,NaN,NaN,NaN,1543.0,1580.99,...,0.0000,99.3008,0.0040,0.0000,0.0000,0.0000,0.0000,14.0,0.1,NaN
94789,NaN,-331H,18.0,NaN,NaN,NaN,NaN,NaN,1158.0,1153.54,...,0.0320,0.1640,68.9228,1.3796,22.1116,0.8940,2.7900,25.0,1.4,NaN


In [25]:
# Joining method two: we add a zero-occurrence column to the covariate DataFrame
# and then inner join.
cov_samp3 = cov_samp
cov_samp3['zenaida_macroura'] = 0
pd.concat([cov_samp3, zenaida_macroura_df], join = 'inner')

,bailey_ecoregion,bcr,elev_gt,elev_ned,housing_density,housing_percent_vacant,nlcd2001_fs_c11_7500_pland,nlcd2001_fs_c12_7500_pland,nlcd2001_fs_c21_7500_pland,nlcd2001_fs_c22_7500_pland,...,nlcd2006_fs_c43_7500_pland,nlcd2006_fs_c52_7500_pland,nlcd2006_fs_c71_7500_pland,nlcd2006_fs_c81_7500_pland,nlcd2006_fs_c82_7500_pland,nlcd2006_fs_c90_7500_pland,nlcd2006_fs_c95_7500_pland,omernik_l3_ecoregion,pop00_sqmi,zenaida_macroura
67743,-212F,13.0,310.0,310.55,19.574237,0.061856,0.1108,0.0,4.5652,0.6076,...,7.2500,5.9248,0.7676,31.5032,21.9404,4.3096,0.2968,83.0,49.8,0
76056,-331G,17.0,972.0,982.21,0.194566,0.169435,0.0148,0.0,0.0000,0.0000,...,0.0000,34.0616,63.6876,0.0000,1.0732,0.5096,0.3612,43.0,0.5,0
97231,M221D,28.0,91.0,104.22,29.660421,0.111111,4.2388,0.0,7.8680,2.3204,...,1.5828,0.0000,0.0068,38.7612,2.8116,0.7840,0.0860,66.0,72.9,0
103640,-331G,17.0,1218.0,1231.32,0.452639,0.146552,0.1460,0.0,0.3980,0.0144,...,0.0000,2.5516,49.2472,0.0628,41.9392,2.4564,0.0204,43.0,1.0,0
39946,-222E,24.0,211.0,210.74,39.576840,0.044566,0.1528,0.0,4.0260,0.5216,...,0.3876,0.0380,5.2112,22.7740,1.1776,0.0000,0.0180,71.0,105.4,0
8414,-221H,28.0,549.0,542.01,20.502175,0.085450,0.2128,0.0,2.6180,0.5684,...,6.8400,5.6288,7.4204,6.7540,0.7512,0.3672,0.0000,68.0,46.5,0
60517,-234A,27.0,80.0,82.58,40.498268,0.076923,0.8364,0.0,6.7872,6.0896,...,0.3740,9.2892,0.0716,11.3528,37.8564,6.4500,1.1856,74.0,100.2,0
40903,M331F,16.0,1889.0,1900.21,3.024832,0.252662,0.0152,0.0,0.0608,0.0024,...,0.0212,42.1576,18.5436,0.3608,0.0000,0.5536,0.0204,21.0,5.5,0
50351,-322A,33.0,1543.0,1580.99,0.110227,0.326923,0.0000,0.0,0.4988,0.1228,...,0.0000,99.3008,0.0040,0.0000,0.0000,0.0000,0.0000,14.0,0.1,0
94789,-331H,18.0,1158.0,1153.54,0.667717,0.180451,0.9756,0.0,1.8836,0.5896,...,0.0320,0.1640,68.9228,1.3796,22.1116,0.8940,2.7900,25.0,1.4,0


Now *that's* the kind of thing we can give to `scikit-learn`.

Outstanding questions:

- Why are some covariates missing from each dataset? Which ones are included in the migrations table and which aren't?
- Is it more correct to include *all* of the migrations table as well as the background covariates, or should I only subsample the observations of a bird and compare that to the background covariates?

I'll research these tomorrow with some lit review.